### Simple Decision Tree from Scratch using Entropy and Information Gain

In [1]:
import pandas as pd
import numpy as np

In [2]:
class Node:
    def __init__(self, value):
        self.value = value     # stores the name of the column from df
        self.categories = []   # stores the unique values for a column
        self.next = []         # stores the childs of this node

In [3]:
# CREATE DECISION TREE

root = None

def calcEntropy(p):
    return -p*np.log2(p)


# Select the best feature based on Entropy and Information gain
def select_best_feature(df, targetClassName):
    columns = []
    igPerColumn = []
    
    # calculate entropy and information gain
    for col in df.drop(targetClassName, axis = 1):
        information_gain = 0
        for category in df[col].unique():
            entropy = 0
            
            # this loop calculates entropy for different classes for particular category
            for cls in df[targetClassName].unique():
                numerator = df[col][ (df[col] == category) & (df[targetClassName] == cls) ].count()
                denominator = df[col][df[col] == category].count()
                
                # calculate entropies
                if numerator == 0:
                    entropy += 0
                else:
                    entropy += calcEntropy(numerator/denominator)
            
            # calculate information gain for that particular categorical value in that column
            # add into same variable for all the categories in that column.
            information_gain += ( df[col][df[col] == category].count() / df[col].count() ) * (entropy)
            
        # append column name and information gain values to respective lists.
        columns.append(col)
        igPerColumn.append(1 - information_gain)
    
    # returns the column name with highest information gain
    return columns[np.argmax(igPerColumn)]


# Builds the tree recursively.
def make_tree(df, targetClassName):
    
    global root
    # initialize root.
    if root == None:
        best_feature = select_best_feature(df, targetClassName)
        root = Node(best_feature)
        node = root
    
    else:
        best_feature = select_best_feature(df, targetClassName)
        node = Node(best_feature)
        
    # Create the leaf of that node : LOGIC : if only one class is fetched for the new df : it is the leaf.
    # if only one class is found for the dataframe i.e, leaf node is found
    if len(df[targetClassName].unique()) == 1:
        return df[targetClassName].unique()[0]
    
    
    
    for category in df[best_feature].unique():
        
        # append the category value for this node
        node.categories.append(category)
        
        # create a new dataframe with selected category value for the best feature and then remove the best fetaure.
        new_df = df[df[best_feature] == category].drop(best_feature, axis = 1)
        
        # recursively create the whole tree
        node.next.append(make_tree(new_df, targetClassName))
        
    return node

In [4]:
# PREDICTION FUNCTIONALITY

# main function to start the tree creation  
def fit(df, targetClassName):
    global root
    root = None
    make_tree(df, targetClassName)


# prediction helper function
def prediction_maker(node, df):
    
    # if the node is a lef node : i.e., it will contain the class label only, and its type will be the class type and not Node.
    # if leaf node is found, Return it.
    if type(node) != type(Node('a')):
        return node
    else:
        category = df[node.value].values[0]        # Get the category value for the current nodes value (column name).
        
        # apply the filters to the dataframe and obtain the filtered out dataframe.
        new_df = df[df[node.value] == category].drop(node.value, axis = 1)
        idx = None                                 # this will be used for indexing the next (child) value for that category.
        for i in range(len(node.categories)):
            if category == node.categories[i]:
                idx = i
                
        # Cecursively find the leaf node.
        ret_class = prediction_maker(node.next[idx], new_df)
        return ret_class
                    
    
# function created for prediction
def predict(df):
    global root
    pred = prediction_maker(root, df)
    return pred

In [5]:
df = pd.DataFrame([['sunny',    'hot',   'high', 'false',  'no'],
                     ['sunny',    'hot',   'high',  'true',  'no'],
                     ['overcast', 'hot',   'high', 'false', 'yes'],
                     ['rainy',   'mild',   'high', 'false', 'yes'],
                     ['rainy',   'cool', 'normal', 'false', 'yes'],
                     ['rainy',   'cool', 'normal',  'true',  'no'],
                     ['overcast','cool', 'normal',  'true', 'yes'],
                     ['sunny',   'mild',   'high', 'false',  'no'],
                     ['sunny',   'cool', 'normal', 'false', 'yes'],
                     ['rainy',   'mild', 'normal', 'false', 'yes'],
                     ['sunny',   'mild', 'normal',  'true', 'yes'],
                     ['overcast','mild',   'high',  'true', 'yes'],
                     ['overcast', 'hot', 'normal', 'false', 'yes'],
                     ['rainy',   'mild',   'high',  'true',  'no']],
                     columns = ['outlook', 'temperature', 'humidity', 'windy', 'play'])

# fit the data to the tree
fit(df, 'play')

In [6]:
# Make Predictions

out = predict(df = pd.DataFrame([['sunny',    'hot',   'high', 'false']],
                     columns = ['outlook', 'temperature', 'humidity', 'windy']))
print("play : ", out)

out = predict(df = pd.DataFrame([['rainy',   'mild',   'high', 'false']],
                     columns = ['outlook', 'temperature', 'humidity', 'windy']))
print("play : ", out)

play :  no
play :  yes
